# 분석용 데이터 생성

데이터 다운로드 링크

https://www.kaggle.com/c/kkbox-churn-prediction-challenge/data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

### 데이터 불러오기

In [ ]:
data_dir = 'C:\\KIMUJUNG\\team_project\\data\\'
df_members = pd.read_csv(data_dir + 'members_v3.csv')
train_v1 = pd.read_csv(data_dir + 'train.csv')
train_v2 = pd.read_csv(data_dir + 'train_v2.csv')
transactions_v1 = pd.read_csv(data_dir + 'transactions.csv')
transactions_v2 = pd.read_csv(data_dir + 'transactions_v2.csv')

### ID 변수인 msno 바꿔주기

In [ ]:
df_member = pd.read_csv('C:/ex_data/members_v3.csv')
df_member_no = df_member[['msno']].copy()
df_member_no.head()

In [ ]:
label_encoder = LabelEncoder()
df_member_no['msno_encoded'] = label_encoder.fit_transform(df_member_no['msno']) + 1
df_member_no.head()

In [ ]:
# members_encoded.csv 파일 로드
df_members = pd.read_csv('C:/ex_data/members_encoded.csv')

# msno와 msno_encoded 컬럼만 추출하여 매핑 만들기
msno_mapping = dict(zip(df_members['msno'], df_members['msno_encoded']))
msno_mapping

In [ ]:
unique_label = pd.read_csv(data_dir + 'members_encoded.csv')

### 로그 파일 전처리

In [ ]:
# user_logs.csv 파일 로드
df_user_logs = pd.read_csv('C:/ex_data/user_logs_v2.csv')

# msno 컬럼을 msno_encoded로 변환
df_user_logs['msno'] = df_user_logs['msno'].map(msno_mapping).astype('Int64')

# csv 저장
df_user_logs.to_csv('C:/ex_data/user_logs_v2_encoded.csv', index=False)

In [ ]:
df_user_logs_v2_encoded = pd.read_csv('C:/ex_data/user_logs_v2_encoded.csv')
df_user_logs_v2_encoded.info(), df_user_logs_v2_encoded.describe()

### user_log.csv 처리 (chunk로 분할 처리)

In [ ]:
# 대용량 CSV를 분할 처리할 크기 (메모리 16기가면 백만, 32기가면 3백만 까지 여유로움)
chunk_size = 1_000_000  
output_file = "C:/ex_data/user_logs_encoded.csv"

# CSV를 append 모드로 저장하기 위해 처음 파일 생성
first_chunk = True  

# 대용량 CSV 파일을 한 번에 다 읽지 않고 부분적으로 처리
for chunk in pd.read_csv("C:/ex_data/user_logs.csv", chunksize=chunk_size):
    chunk['msno'] = chunk['msno'].map(msno_mapping).astype('Int64')  # msno를 정수형으로 변환

    # 변환된 데이터를 CSV 파일로 저장 (첫 번째 반복에서는 헤더 포함, 이후에는 생략)
    chunk.to_csv(output_file, index=False, mode='w' if first_chunk else 'a', header=first_chunk)
    
    first_chunk = False  # 이후에는 'append' 모드로 저장

### 2GB씩 분할

In [ ]:
# 판다스에서 실행할려면 파일 크기가 2GB 제한
chunk_size = 50_000_000  
file_index = 1  # 파일 번호

for chunk in pd.read_csv("C:/ex_data/user_logs_encoded.csv", chunksize=chunk_size):
    output_file = f"C:/ex_data/user_logs_encoded_part{file_index}.csv"
    chunk.to_csv(output_file, index=False)

    print(f"파일 저장 완료: {output_file}")  # 저장 완료 메시지 출력
    file_index += 1  # 다음 파일 번호 증가

print('파일 분할 완료!')

In [ ]:
# 변환할 파일 리스트
file_list = [f"C:/ex_data/user_logs_encoded_part{i}.csv" for i in range(1, 9)]

for file in file_list:
    # CSV 파일 읽기
    df = pd.read_csv(file)

    # msno 컬럼을 Int64로 변환
    df['msno'] = df['msno'].astype('Int64')

    # 다시 저장 (덮어쓰기)
    df.to_csv(file, index=False)

    print(f"변환 완료: {file}")

print("모든 파일의 msno를 Int64로 변환 완료!")

In [ ]:
split_df = pd.read_csv("C:/ex_data/user_logs_encoded_part7.csv")
split_df.info()

### 병합

In [ ]:
import glob

# 파일 경로 리스트 (user_logs_v2_encoded.csv + user_logs_encoded_part*.csv 포함)
file_list = ["C:/ex_data/user_logs_v2_encoded.csv"] + glob.glob("C:/ex_data/user_logs_encoded_part*.csv")

# msno별 데이터를 저장할 딕셔너리 (메모리 절약)
aggregated_data = {}

# 각 파일을 청크 단위로 읽어서 그룹화 & 누적
for file in file_list:
    print(f"파일 처리 중: {file}")

    # CSV 파일 읽기 (청크 단위 처리)
    for chunk in pd.read_csv(file, chunksize=3_000_000):  # 청크 크기 조절 가능
        # msno 기준으로 그룹화하여 데이터 합산
        grouped = chunk.groupby("msno").agg(
            num_25=("num_25", "sum"),
            num_50=("num_50", "sum"),
            num_75=("num_75", "sum"),
            num_985=("num_985", "sum"),
            num_100=("num_100", "sum"),
            num_unq=("num_unq", "sum"),
            total_secs=("total_secs", "sum"),
            log_start=("date", "min"),  # 가장 빠른 날짜
            log_end=("date", "max")  # 가장 마지막 날짜
        ).reset_index()

        # 🏗 기존 데이터와 병합하여 누적 (딕셔너리를 활용한 병합)
        for row in grouped.itertuples(index=False):
            msno = row.msno
            if msno in aggregated_data:
                aggregated_data[msno]["num_25"] += row.num_25
                aggregated_data[msno]["num_50"] += row.num_50
                aggregated_data[msno]["num_75"] += row.num_75
                aggregated_data[msno]["num_985"] += row.num_985
                aggregated_data[msno]["num_100"] += row.num_100
                aggregated_data[msno]["num_unq"] += row.num_unq
                aggregated_data[msno]["total_secs"] += row.total_secs
                aggregated_data[msno]["log_start"] = min(aggregated_data[msno]["log_start"], row.log_start)
                aggregated_data[msno]["log_end"] = max(aggregated_data[msno]["log_end"], row.log_end)
            else:
                aggregated_data[msno] = {
                    "num_25": row.num_25,
                    "num_50": row.num_50,
                    "num_75": row.num_75,
                    "num_985": row.num_985,
                    "num_100": row.num_100,
                    "num_unq": row.num_unq,
                    "total_secs": row.total_secs,
                    "log_start": row.log_start,
                    "log_end": row.log_end
                }

# 딕셔너리를 DataFrame으로 변환 (msno를 컬럼으로 유지)
df_final = pd.DataFrame.from_dict(aggregated_data, orient="index")

# msno를 일반 컬럼으로 유지하고 인덱스 제거
df_final.index.name = "msno"
df_final.reset_index(inplace=True)

# CSV 파일로 저장 (인덱스 없이)
df_final.to_csv("C:/ex_data/user_logs_summary.csv", index=False)

print("저장 완료")

In [ ]:
log_data = pd.read_csv(data_dir + 'user_logs_summary.csv')

### train data 전처리

In [ ]:
train_v1['version'] = 'v1'
train_v2['version'] = 'v2'
train = pd.concat([train_v1, train_v2])
train = train.sort_values(['msno', 'version'], ascending = [True, False]).reset_index(drop=True)

In [ ]:
aa = train['msno'][(train['is_churn'] == 0) & (train['version'] == 'v2')]
bb = train['msno'][(train['is_churn'] == 1) & (train['version'] == 'v1')]
cc = list(set(aa) & set(bb))
train['is_back'] = train['msno'].isin(cc).astype(int)

In [ ]:
# 중복값을 제거하되, 중복 Row 중 첫번째 Row를 남기는 옵션을 선택함
train.drop_duplicates(subset='msno', keep='first', inplace=True)
train.query('msno in "+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o="')
# version 컬럼 제거
train = train.drop('version', axis=1)

### transaction 데이터 처리

In [ ]:
# 구매 기록 데이터 합치기
transactions = pd.concat([transactions_v1, transactions_v2], ignore_index=True)
transactions = transactions.sort_values(['msno', 'transaction_date']).reset_index(drop=True)

In [ ]:
# 0으로 나누는 오류 방지: plan_list_price가 0인 경우는 NaN 처리
transactions["discount_rate"] = np.where(
    transactions["plan_list_price"] != 0, 
    1 - (transactions["actual_amount_paid"] / transactions["plan_list_price"]),
    np.nan  # 원래 가격이 0이면 NaN (이후 평균 계산 시 자동 제외됨)
)

# 사용자 ID(msno) 기준으로 그룹화하여 새로운 데이터프레임 생성
df_transaction = transactions.groupby("msno").agg(
    payment_plan_sum=("payment_plan_days", "sum"),
    plan_list_price=("plan_list_price", "sum"),
    actual_amount_paid=("actual_amount_paid", "sum"),
    discount_rate=("discount_rate", "mean"),  # 개별 거래별 할인율 평균
    is_auto_renew=("is_auto_renew", "mean"),
    membership_expire_date=("membership_expire_date", "max"),
    is_cancel=("is_cancel", "mean"),
    transaction_count=("msno", "count")
).reset_index()

In [ ]:
# 기존 데이터프레임에 msno_num 추가
train_en = train.merge(unique_label, on='msno', how='inner').drop(columns=['msno'])
members_en = df_members.merge(unique_label, on='msno', how='inner').drop(columns=['msno'])
df_transaction_en = df_transaction.merge(unique_label, on='msno', how='inner').drop(columns=['msno'])

In [ ]:
kkbox_transaction_merge = pd.merge(train_en, df_transaction_en, on='msno_encoded', how='inner',)
kkbox_merge = pd.merge(members_en, kkbox_transaction_merge, on='msno_encoded', how='inner')
kkbox_merge['msno'] = kkbox_merge['msno_encoded']
kkbox_merge = kkbox_merge[['msno', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time',
        'is_churn', 'is_back', 'payment_plan_sum',
        'plan_list_price', 'actual_amount_paid', 'discount_rate',
        'is_auto_renew', 'membership_expire_date', 'is_cancel', 'transaction_count']]

In [ ]:
kkbox_merge_final = pd.merge(kkbox_merge, log_data, on='msno', how='inner')
kkbox_merge_final.to_csv(data_dir + "kkbox_data_total.csv", index=False)

### 성별 전처리

In [ ]:
df = pd.read_csv(data_dir + 'kkbox_data_total.csv')

In [ ]:
def duration(df):
    # membership_expire_date와 registration_init_time은 'YYYY-MM-DD' 형식으로 되어 있으므로, 그에 맞춰 변환
    df['membership_expire_date'] = pd.to_datetime(df['membership_expire_date'], errors='coerce')
    df['registration_init_time'] = pd.to_datetime(df['registration_init_time'], errors='coerce')
    
    # log_start와 log_end는 'YYYYMMDD' 형식이므로, 그에 맞춰 변환
    df['log_start'] = pd.to_datetime(df['log_start'].astype(str), format='%Y%m%d', errors='coerce')
    df['log_end'] = pd.to_datetime(df['log_end'].astype(str), format='%Y%m%d', errors='coerce')

    # 등록기간 계산: membership_expire_date와 registration_init_time 차이
    df['registration_duration'] = (df['membership_expire_date'] - df['registration_init_time']).dt.days

    # 음악 청취 기간 계산: log_end와 log_start 차이
    df['listening_duration'] = (df['log_end'] - df['log_start']).dt.days

    # 필요 없는 컬럼 삭제
    df.drop(['membership_expire_date', 'registration_init_time', 'log_start', 'log_end'], axis=1, inplace=True)

    return df

# 함수 실행
df = duration(df)

In [ ]:
df['gender'].isna().sum()

In [ ]:
df = df[df['gender'].notna()]

In [ ]:
df['gender'].isna().sum()

In [ ]:

df.info()

In [ ]:
df['gender']

In [ ]:
def encode_gender(df):
    # 성별 인코딩: M -> 1, F -> 0, nan -> -1 (또는 원하는 값으로 변경)
    df['gender'] = df['gender'].map({'male': 1, 'female': 0})
    
    return df

In [ ]:
df = encode_gender(df)
df['gender'].unique()

### 나이 전처리

In [ ]:

df["bd"] = df["bd"].abs()

In [ ]:
df = df[(df["bd"] >= 10) & (df["bd"] <= 100)]

In [ ]:
df.drop(['Unnamed: 0', 'is_back', 'msno'], axis=1, inplace=True)

### 이상치 제거

In [ ]:
aa = df[df['payment_plan_sum'].isin([0, 2882, 3550])].index
df.drop(aa, inplace=True)
bb = df[df['plan_list_price'].isin([0, 13186, 16800, 17433])].index
df.drop(bb, inplace=True)
cc = df[df['plan_list_price'].isin([0, 13186, 16800, 17433])].index
df.drop(cc, inplace=True)

In [ ]:
aa = df[df['num_25'] >= 20000].index
df.drop(aa, inplace=True)
bb = df[df['num_50'] >= 30000].index
df.drop(bb, inplace=True)
cc = df[df['num_75'] >= 10000].index
df.drop(cc, inplace=True)
dd = df[df['num_985'] >= 20000].index
df.drop(dd, inplace=True)
ee = df[df['num_100'] >= 35000].index
df.drop(ee, inplace=True)
ff = df[df['num_unq'] >= 24000].index
df.drop(ff, inplace=True)
gg = df[df['total_secs'] < 0].index
df.drop(gg, inplace=True)
df.drop(df[df['msno'] == 4524786].index, inplace=True)

### 최종 데이터 셋

In [ ]:
df.to_csv("Real_Total_Data.csv", index=False)